# `IsotropicGasID`: An Einstein Toolkit Initial Data Thorn for isotripic gas initial data
## Author: Leo Werneck
### Formatting improvements courtesy Brandon Clark

**Notebook Status:** <font color='red'><b> In progress </b></font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This notebook is organized as follows

1. [Step 1](#initialize_nrpy): Initialize core NRPy+/Python modules

1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initialize_nrpy'></a>

# Step 1: Initialize core NRPy+/Python modules \[Back to [top](#toc)\]
$$\label{initialize_nrpy}$$

Let us start by loading the core NRPy+ and Python modules that we will need to write down our C code.

In [1]:
# Step 1: Initialize core NRPy+ and Python modules
import os,sys,shutil           # Standard Python modules for multiplatform OS-level functions
import sympy as sp             # SymPy: Symbolic algebra package for Python (NRPy+'s backbone)
sys.path.append(os.path.join("nrpy_core"))
import outputC as outC         # NRPy+: Core C output functions
import NRPy_param_funcs as par # NRPy+: Parameter interface
import cmdline_helper as cmd   # NRPy+: Multi-platform Python command-line interface
import grid as gri             # NRPy+: Numerical grids interface
import indexedexp as ixp       # NRPy+: Indexed expression support

# Step 1.a: Set thorn name
ThornName = "IsotropicGasID"

# Step 1.b: Create the thorn directory
Thorndir = os.path.join(ThornName)
if os.path.exists(Thorndir):
    shutil.rmtree(Thorndir)
cmd.mkdir(Thorndir)

# Step 1.c: Create the source code directory
Srcdir = os.path.join(Thorndir,"src")
cmd.mkdir(Srcdir)

# Step 1.d: Set precision of the code to CCTK_REAL
par.set_parval_from_str("outputC::PRECISION","CCTK_REAL")

# Step 1.e: Set gridfunction memory access to ETK
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

<a id='id_symbolic'></a>

# Step 2: Symbolic expressions for the initial data \[Back to [top](#toc)\]
$$\label{id_symbolic}$$

We now generate symbolic expressions for the spacetime and hydrodynamics initial data:

$$
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\begin{aligned}
\alpha = 0,\ \beta^{i} = 0&,\ \gamma_{ij} = \delta_{ij},\ k_{ij} = 0\\
\rhob = \left(\rhob\right)_{\rm input},\ \ye &= \left(\ye\right)_{\rm input},\ T = T_{\rm input},\\
v^{i} = 0&,\ B^{i} = 0.
\end{aligned}
$$

To make it easier for users to adopt this thorn, we allow for *any* spacetime/hydrodynamics gridfunctions within the Einstein Toolkit to be used.

In [2]:
# Step 2: Symbolic expressions for the initial data
# Step 2.a: Declare basic constants
zero = sp.sympify(0)
one  = sp.sympify(1)

# Step 2.b: Initialize list of gridfunctions & expressions
vars_list = []
expr_list = []

# Step 2.c: Thorn ID parameters
ID_Ye,ID_rhob,ID_T = par.Cparameters("REAL",ThornName,
                                     [ThornName+"_Y_e",ThornName+"_rho",ThornName+"_temperature"],
                                     [     "-1"      ,      "-1"      ,     "-1"     ])

# Step 2.c: Symbolic expressions for the ID
vars_list.extend(["alp",  "Y_e", "rho", "temperature", "press", "eps"])
expr_list.extend([  one  ,ID_Ye,ID_rhob,ID_T,"IsotropicGasID_press","IsotropicGasID_eps"])
for i in range(3):
    vars_list.append("beta"+chr(ord('x')+i))
    expr_list.append(zero)
    vars_list.append("vel"+chr(ord('x')+i))
    expr_list.append(zero)
    for j in range(i,3):
        vars_list.append("g"+chr(ord('x')+i)+chr(ord('x')+j))
        if j==i:
            expr_list.append(one)
        else:
            expr_list.append(zero)
        vars_list.append("k"+chr(ord('x')+i)+chr(ord('x')+j))
        expr_list.append(zero)

# Step 2.d: Sort the lists
vars_list,expr_list = [list(x) for x in zip(*sorted(zip(vars_list,expr_list), key=lambda pair: pair[0]))]

# Step 2.e: Generate the ID & gf pointer strings
N = len(vars_list)
ID_string = ""
GF_pointers_string = ""
indent = "        "
for n in range(N):
    gf    = vars_list[n]
    value = expr_list[n]
    ID_string += indent+gf+"[idx] = "+str(value)+";\n"

<a id='driver_function'></a>

# Step 3:  `IsotropicGasID` - the thorn's driver function \[Back to [top](#toc)\]
$$\label{driver_function}$$

Below we implement the thorn's driver function, `IsotropicGasID`, which is responsible for setting up the desired initial data.

In [3]:
def add_to_Cfunction_dict__IsotropicGasID():
    desc = """
(c) 2021 Leo Werneck

This is the thorn's driver function, responsible
for setting the initial data to that of an isotropic
gas of constant density, temperature, and electron
fraction in Minkowski space.
"""
    includes = ["cctk.h","cctk_Arguments.h","cctk_Parameters.h"]
    prefunc  = """
#define velx (&vel[0*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])
#define vely (&vel[1*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])
#define velz (&vel[2*cctk_lsh[0]*cctk_lsh[1]*cctk_lsh[2]])
"""
    c_type   = "void"
    name     = "IsotropicGasID"
    params   = "CCTK_ARGUMENTS"
    body     = r"""
  // Step 1: Get access to gridfunctions and parameters
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  // Step 2: Compute local pressure and epsilon
  CCTK_REAL IsotropicGasID_press, IsotropicGasID_eps;
  WVU_EOS_P_and_eps_from_rho_Ye_T( IsotropicGasID_rho,
                                   IsotropicGasID_Y_e,
                                   IsotropicGasID_temperature,
                                  &IsotropicGasID_press,
                                  &IsotropicGasID_eps );

  // Step 3: Loop over the grid and set the ID
#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++) {
    for(int j=0;j<cctk_lsh[1];j++) {
      for(int i=0;i<cctk_lsh[0];i++) {
      
        const int idx = CCTK_GFINDEX3D(cctkGH,i,j,k);

"""+ID_string+"""

      }
    }
  }
"""
    loopopts = ""
    outC.outCfunction(os.path.join(Srcdir,name+".c"),
                      includes=includes,prefunc=prefunc,desc=desc,c_type=c_type,
                      name=name,params=params,body=body,enableCparameters=False)
#     outC.add_to_Cfunction_dict(includes=includes,prefunc=prefunc,desc=desc,c_type=c_type,
#                                name=name,params=params,body=body,enableCparameters=False)

<a id='configuration_files'></a>

# Step 4: Thorn configuration files \[Back to [top](#toc)\]
$$\label{configuration_files}$$

We now generate all the configuration files required by our thorn. These are:
1. `interface.ccl`: contains information about how the thorn interfaces with the rest of the toolkit;
1. `param.ccl`: contains all the parameters defined by this thorn;
1. `schedule.ccl`: specifies when the functions in these thorn are called by the scheduler;
4. `make.code.defn`: specifies which files need to be compiled.

<a id='interface_ccl'></a>

## Step 4.a: `interface.ccl` \[Back to [top](#toc)\]
$$\label{interface_ccl}$$

In [4]:
# Step 4.a: Write the interface.ccl file
def generate_interface_ccl():
    filepath = os.path.join(Thorndir,"interface.ccl")
    with open(filepath,"w") as file:
        file.write("# "+ThornName+""" interface.ccl file
implements: """+ThornName+"""
inherits: ADMBase grid HydroBase
######################################
### Aliased function from WVU_EOS  ###
######################################
void FUNCTION WVU_EOS_P_and_eps_from_rho_Ye_T( CCTK_REAL IN rho, \\
                                               CCTK_REAL IN Ye,  \\
                                               CCTK_REAL IN T,   \\
                                               CCTK_REAL OUT P,  \\
                                               CCTK_REAL OUT eps )

USES FUNCTION WVU_EOS_P_and_eps_from_rho_Ye_T\n""")
    print("Output interface.ccl file to "+filepath)

<a id='param_ccl'></a>

## Step 4.b: `param.ccl` \[Back to [top](#toc)\]
$$\label{param_ccl}$$

In [5]:
# Step 4.b: Write the param.ccl file
def generate_param_ccl():
    filepath = os.path.join(Thorndir,"param.ccl")
    params_string = "# "+ThornName+""" param.ccl file
shares: grid
shares: ADMBase
USES CCTK_INT lapse_timelevels
USES CCTK_INT shift_timelevels
USES CCTK_INT metric_timelevels

USES KEYWORD metric_type

EXTENDS KEYWORD initial_data
{
  \""""+ThornName+"""\" :: \"ADMBase initial data\"
}

EXTENDS KEYWORD initial_lapse
{
  \""""+ThornName+"""\" :: \"Initial lapse\"
}

EXTENDS KEYWORD initial_shift
{
  \""""+ThornName+"""\" :: \"Initial shift\"
}

shares: HydroBase
USES CCTK_INT timelevels

EXTENDS KEYWORD initial_hydro
{
  \""""+ThornName+"""\" :: \"HydroBase initial data\"
}

EXTENDS KEYWORD initial_Y_e
{
  \""""+ThornName+"""\" :: \"HydroBase initial data\"
}

EXTENDS KEYWORD initial_temperature
{
  \""""+ThornName+"""\" :: \"HydroBase initial data\"
}\n\n"""
    
    for param in par.glb_Cparams_list:
        if param.module == ThornName:
            params_string += "restricted:\n"
            if param.type == 'REAL':
                params_string += "CCTK_REAL "
            elif param.type == 'int':
                params_string += "CCTK_INT "
            params_string += param.parname+" \""+param.parname+"\"\n"
            params_string += "{\n  *:* :: \"Anything goes\"\n} "+param.defaultval+"\n\n"
    with open(os.path.join(Thorndir,"param.ccl"),"w") as file:
        file.write(params_string)
    print("Output param.ccl file to "+filepath)

<a id='schedule_ccl'></a>

## Step 4.c: `schedule.ccl` \[Back to [top](#toc)\]
$$\label{schedule_ccl}$$

In [6]:
def generate_schedule_ccl():
    filepath = os.path.join(Thorndir,"schedule.ccl")
    with open(filepath,"w") as file:
        file.write("# "+ThornName+""" schedule.ccl file
    
STORAGE: ADMBase::metric[metric_timelevels]
STORAGE: ADMBase::curv[metric_timelevels]
STORAGE: ADMBase::lapse[lapse_timelevels]
STORAGE: ADMBase::shift[shift_timelevels]
STORAGE: HydroBase::rho[timelevels]
STORAGE: HydroBase::press[timelevels]
STORAGE: HydroBase::eps[timelevels]
STORAGE: HydroBase::vel[timelevels]
STORAGE: HydroBase::Y_e[timelevels]
STORAGE: HydroBase::temperature[timelevels]
STORAGE: HydroBase::entropy[timelevels]

schedule IsotropicGasID IN HydroBase_Initial
{
  LANG: C
  WRITES: ADMBase::alp(Everywhere)
  WRITES: ADMBase::betax(Everywhere)
  WRITES: ADMBase::betay(Everywhere)
  WRITES: ADMBase::betaz(Everywhere)
  WRITES: ADMBase::kxx(Everywhere)
  WRITES: ADMBase::kxy(Everywhere)
  WRITES: ADMBase::kxz(Everywhere)
  WRITES: ADMBase::kyy(Everywhere)
  WRITES: ADMBase::kyz(Everywhere)
  WRITES: ADMBase::kzz(Everywhere)
  WRITES: ADMBase::gxx(Everywhere)
  WRITES: ADMBase::gxy(Everywhere)
  WRITES: ADMBase::gxz(Everywhere)
  WRITES: ADMBase::gyy(Everywhere)
  WRITES: ADMBase::gyz(Everywhere)
  WRITES: ADMBase::gzz(Everywhere)
  WRITES: HydroBase::vel[0](Everywhere)
  WRITES: HydroBase::vel[1](Everywhere)
  WRITES: HydroBase::vel[2](Everywhere)
  WRITES: HydroBase::rho(Everywhere)
  WRITES: HydroBase::eps(Everywhere)
  WRITES: HydroBase::press(Everywhere)
  WRITES: HydroBase::Y_e(Everywhere)
  WRITES: HydroBase::temperature(Everywhere)
} \"Set up general relativistic hydrodynamic (GRHD) fields for isotropic gas initial data\"""")
    print("Output schedule.ccl file to "+filepath)

<a id='make_code_defn'></a>

## Step 4.d: `make.code.defn` \[Back to [top](#toc)\]
$$\label{make_code_defn}$$

In [7]:
def generate_make_code_defn():
    filepath = os.path.join(Srcdir,"make.code.defn")
    with open(filepath,"w") as file:
        file.write("# "+ThornName+""" make.code.defn file
SRCS = IsotropicGasID.c""")
    print("Output make.code.defn file to "+filepath)

In [8]:
def generate_ccl_files():
    generate_interface_ccl()
    generate_param_ccl()
    generate_schedule_ccl()
    generate_make_code_defn()

In [9]:
def generate_thorn__IsotropicGasID():
    generate_ccl_files()
    add_to_Cfunction_dict__IsotropicGasID()

In [10]:
generate_thorn__IsotropicGasID()

Output interface.ccl file to IsotropicGasID/interface.ccl
Output param.ccl file to IsotropicGasID/param.ccl
Output schedule.ccl file to IsotropicGasID/schedule.ccl
Output make.code.defn file to IsotropicGasID/src/make.code.defn
Output C function IsotropicGasID() to file IsotropicGasID/src/IsotropicGasID.c


<a id='latex_pdf_output'></a>

# Step n: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf](Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Tutorial-ETK_thorn-IsotropicGasID")

Created Tutorial-Tutorial-ETK_thorn-IsotropicGasID.tex, and compiled LaTeX
    file to PDF file Tutorial-Tutorial-ETK_thorn-IsotropicGasID.pdf
